### List Functions

In [2]:
from rope.base.project import Project

In [3]:
project = Project('.')

In [14]:
mod = project.get_resource("file_with_funcs.py")
mod

In [8]:
from rope.base import libutils

In [9]:
libutils.analyze_module(project, mod)

In [13]:
pymod = libutils.get_string_module(project, """def power_func(num, power):
    return num**power

def square(num):
    return power_func(num, 2)
""")
pymod

In [21]:
# get members

In [29]:
from inspect import getmembers, isfunction

In [30]:
import file_with_funcs

In [31]:
dir(file_with_funcs)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'power_func',
 'square']

In [24]:
res = getmembers(file_with_funcs, isfunction)[0][1]

In [27]:
res

<function file_with_funcs.power_func(num, power)>

In [32]:
import inspect

In [33]:
inspect.getsource(res)

'def power_func(num, power):\n    return num**power\n'

In [34]:
inspect.getsourcelines(res)

(['def power_func(num, power):\n', '    return num**power\n'], 1)

In [ ]:
# caveats: have to be a python object

# workaround
# write string to a file vs eval
# imports it
# do inspect

### Finding Functions

In [ ]:
# @task
# def move_class(ctxt, class_name, source, target, do=False):
#     """
#     Move class: --class-name <> --source <module> --target <module> [--do False]
#     """
#     finder = Finder(PROJECT, class_name)
#     source_resource = PROJECT.get_resource(source)
#     target_resource = PROJECT.get_resource(target)
#     class_occurrence = next(
#         occ for occ in finder.find_occurrences(resource=source_resource)
#         if occ.is_defined())
#     mover = move.create_move(PROJECT, source_resource, class_occurrence.offset)
#     changes = mover.get_changes(target_resource)
#     PROJECT.do(changes)

In [41]:
# find_occurrences
# find_implementations
# find_definition

In [46]:
from rope.refactor import occurrences
from rope.refactor.occurrences import Finder

In [74]:
from rope.base.resources import File
from pathlib import Path

In [76]:
file = File(project, "new_file.py")
nested_file = File(project, "path/to/new_file.py")
print(file.exists())
print(nested_file.exists())
# error if already exists
if not file.exists():
    file.create()
# error if parent or nested folder doesn't exist: must combine with Python's Path
if not nested_file.parent.exists():
    Path(nested_file.parent.path).mkdir(parents=True, exist_ok=True)
    
nested_file.create()
print(nested_file.parent.exists())

print(file.exists())
print(nested_file.exists())

True
False
True
True
True


In [ ]:
# note: check if file not exist

In [110]:
finder = Finder(project, "square")
src_res = project.get_resource("file_with_funcs.py")
dest_res = project.get_resource("new_file.py")

### Examples
https://github.com/python-rope/rope/issues/231

#### Changes
https://github.com/python-rope/rope/blob/8eb79a22009b272dfd5a3a90d031c9c2c52311fe/rope/base/change.py

#### PyModule
https://github.com/python-rope/rope/blob/ede9464b5c887e9314f3c375e7fb88a1b04f9d77/rope/base/pyobjectsdef.py

* can infer if the code has errors

#### PyFunction
https://github.com/python-rope/rope/blob/d6f809e41d26a9edfb60a74906964078a1c20e7d/rope/base/pyobjects.py

#### PyName
https://github.com/python-rope/rope/blob/221dec95844f00c37150adafcd51a5f0f371399f/rope/base/pynames.py

### Finding Occurences with Rope

In [78]:
for occ in finder.find_occurrences(resource=src_res):
    print(occ)

In [214]:
occ.get_primary_range()

(55, 61)

In [212]:
occ.offset

55

In [98]:
occ.lineno

4

In [92]:
from rope.base.pynames import DefinedName

In [97]:
?DefinedName

Init signature: DefinedName(pyobject)
Docstring:      References to `PyObject` inside python programs
File:           ~/universe-venv/lib/python3.9/site-packages/rope/base/pynames.py
Type:           type
Subclasses:     


In [48]:
# finder = occurrences.create_finder(project, 'file_with_funcs', 'b')
# for occurrence in finder.find_occurrences():
#     print(occurence)

### Moving Functions

In [ ]:
# how to find the offset? use occurence
# how to create the destination resource
    # does it have to exist

In [101]:
from rope.refactor import move

In [38]:
from rope.refactor.move import MoveMethod

In [102]:
mover = move.create_move(project, src_res, occ.offset)

In [112]:
changes = mover.get_changes(dest_res)
project.do(changes)

In [145]:
print(changes.get_description())

Moving global <square> - today 15:06:24 :


--- a/file_with_funcs.py
+++ b/file_with_funcs.py
@@ -1,5 +1,3 @@
 def power_func(num, power):
     return num**power
 
-def square(num):
-    return power_func(num, 2)

--- a/new_file.py
+++ b/new_file.py
@@ -0,0 +1,5 @@
+from file_with_funcs import power_func
+
+
+def square(num):
+    return power_func(num, 2)




In [109]:
changes.changes[0]

In [39]:
mover = MoveMethod(project, mod)
# project.do(mover.get_changes(destination))

TypeError: __init__() missing 1 required positional argument: 'offset'

In [ ]:
get_definition_location

In [114]:
py_mod = libutils.get_string_module(project, """def power_func(num, power):
    return num**power

def square(num):
    return power_func(num, 2)""")

In [127]:
py_mod.get_type()

In [128]:
py_mod.get_scope()

In [117]:
py_mod.get_attributes()

{'power_func': <rope.base.pynames.DefinedName at 0x118d5b670>,
 'square': <rope.base.pynames.DefinedName at 0x117d07b50>}

In [133]:
ast_mod = py_mod.get_ast()

In [143]:
ast_mod.body

[<ast.FunctionDef at 0x118d30880>, <ast.FunctionDef at 0x118d30400>]

In [130]:
py_mod.get_ast()

In [124]:
# err_mod = libutils.get_string_module(project, """abc
# def""")
# err_mod.has_errors

### Extract Functions from Code Block

#### Extract
https://github.com/python-rope/rope/blob/master/rope/refactor/extract.py
https://github.com/python-rope/rope/blob/3c8d74dccf97a0c07c4ecd2de42737e37a6e2515/ropetest/refactor/extracttest.py

In [180]:
from rope.refactor.extract import ExtractVariable, ExtractMethod
from rope.refactor import extract

In [150]:
def convert_line_range_to_offset(code, start, end):
    lines = rope.base.codeanalyze.SourceLinesAdapter(code)
    return lines.get_line_start(start), lines.get_line_end(end)

In [151]:
extract_src = project.get_resource("extract_funcs.py")

In [154]:
mod = libutils.get_string_module(project, extract_src.read())

In [172]:
mod.lines.get_line_start(1)

0

In [173]:
mod.lines.get_line_end(mod.lines.length())

82

In [166]:
mod.lines.length()

5

In [181]:
start = mod.lines.get_line_start(1)
end = mod.lines.get_line_end(mod.lines.length())

In [182]:
extractor = extract.ExtractMethod(project, extract_src, start, end)

In [183]:
project.do(extractor.get_changes("repeat_string"))

In [179]:
# extractor = extract.ExtractMethod(self.project, testmod, start, end)
# self.project.do(extractor.get_changes(extracted, **kwds))
# return testmod.read()

In [174]:
from rope.contrib import generate

In [178]:
# create a file
# generate.create_module(project, "create_mod")

### Renaming

https://github.com/python-rope/rope/blob/master/rope/refactor/rename.py

In [184]:
from rope.refactor.rename import Rename

In [209]:
rename_mod = project.get_resource("rename_def.py")

In [210]:
# must know what's the function that is going to be renamed
finder = Finder(project, "abc")

In [211]:
occs = list(finder.find_occurrences(rename_mod))

ModuleSyntaxError: Syntax error in file <rename_def.py> line <1>: invalid syntax

In [198]:
occurence = occs[0]
occurence.offset

4

In [ ]:
rename_mod.fin

In [208]:
# changes = Rename(project, rename_mod, occurence.offset).get_changes('def')

In [205]:
print(changes.get_description())

Renaming <abc> to <def> - today 15:48:39 :


--- a/rename_def.py
+++ b/rename_def.py
@@ -1,2 +1,2 @@
-def abc():
+def def():
     print("hello")



In [207]:
project.do(changes)

In [218]:
class Person:
    @staticmethod
    def call_person(name="person"):
        print(f"hello {name}")
    
    def init_name(self, name):
        self.call_person(name)

In [219]:
Person().init_name("hello")

hello hello
